In [ ]:
import urllib
import xml.etree.ElementTree as ET
import json
from datetime import datetime

In [48]:
url = 'http://export.arxiv.org/api/query?search_query=all:electron&start=0&max_results=1'
response_data = urllib.request.urlopen(url)
xml_data = response_data.read().decode('utf-8')

if response_data.getcode() == 200:
    print('Success')
else:
    print('Failed')

Success


In [49]:
print(xml_data)

<?xml version="1.0" encoding="UTF-8"?>
<feed xmlns="http://www.w3.org/2005/Atom">
  <link href="http://arxiv.org/api/query?search_query%3Dall%3Aelectron%26id_list%3D%26start%3D0%26max_results%3D1" rel="self" type="application/atom+xml"/>
  <title type="html">ArXiv Query: search_query=all:electron&amp;id_list=&amp;start=0&amp;max_results=1</title>
  <id>http://arxiv.org/api/cHxbiOdZaP56ODnBPIenZhzg5f8</id>
  <updated>2024-12-04T00:00:00-05:00</updated>
  <opensearch:totalResults xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">217748</opensearch:totalResults>
  <opensearch:startIndex xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">0</opensearch:startIndex>
  <opensearch:itemsPerPage xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">1</opensearch:itemsPerPage>
  <entry>
    <id>http://arxiv.org/abs/cond-mat/0102536v1</id>
    <updated>2001-02-28T20:12:09Z</updated>
    <published>2001-02-28T20:12:09Z</published>
    <title>Impact of Electron-Electron Cusp on Configur

In [56]:
# Parse the XML data
namespace = {"arxiv": "http://arxiv.org/schemas/atom", "default": "http://www.w3.org/2005/Atom"}
root = ET.fromstring(xml_data)
entry = root.find("default:entry", namespace)

# Convert dates to required formats
updated_date = entry.find("default:updated", namespace).text
create_date = datetime.strptime(updated_date, "%Y-%m-%dT%H:%M:%SZ").strftime("%a, %d %b %Y %H:%M:%S GMT")

# Build the JSON result
result = {
    "id": entry.find("default:id", namespace).text.split("/")[-1],
    "submitter": entry.find("default:author/default:name", namespace).text,
    "authors": ", ".join(author.find("default:name", namespace).text for author in entry.findall("default:author", namespace)),
    "title": entry.find("default:title", namespace).text.strip(),
    "comments": entry.find("arxiv:comment", namespace).text if entry.find("arxiv:comment", namespace) is not None else None,
    "journal-ref": entry.find("arxiv:journal_ref", namespace).text if entry.find("arxiv:journal_ref", namespace) is not None else None,
    "doi": entry.find("arxiv:doi", namespace).text if entry.find("arxiv:doi", namespace) is not None else None,
    "report-no": None,
    "categories": entry.find("arxiv:primary_category", namespace).attrib["term"],
    "license": None,
    "abstract": entry.find("default:summary", namespace).text.strip(),
    "versions": [
        {
            "version": "v1",
            "create": create_date,
        }
    ],
    "update_date": datetime.fromisoformat(root.find("default:updated", namespace).text).strftime("%Y-%m-%d"),
    "authors_parsed": [
        author.find("default:name", namespace).text
        for author in entry.findall("default:author", namespace)
    ]
}

# Convert to JSON
json_result = json.dumps(result, indent=2)
print(json_result)

{
  "id": "0102536v1",
  "submitter": "David Prendergast",
  "authors": "David Prendergast, M. Nolan, Claudia Filippi, Stephen Fahy, J. C. Greer",
  "title": "Impact of Electron-Electron Cusp on Configuration Interaction Energies",
  "comments": "11 pages, 6 figures, 3 tables, LaTeX209, submitted to The Journal of\n  Chemical Physics",
  "journal-ref": "J. Chem. Phys. 115, 1626 (2001)",
  "doi": "10.1063/1.1383585",
  "report-no": null,
  "categories": "cond-mat.str-el",
  "license": null,
  "abstract": "The effect of the electron-electron cusp on the convergence of configuration\ninteraction (CI) wave functions is examined. By analogy with the\npseudopotential approach for electron-ion interactions, an effective\nelectron-electron interaction is developed which closely reproduces the\nscattering of the Coulomb interaction but is smooth and finite at zero\nelectron-electron separation. The exact many-electron wave function for this\nsmooth effective interaction has no cusp at zero elec